# Modelo de Previsão - Acidentes de Trânsito na Paraíba

A previsão de acidentes de trânsito na consiste na aplicação de técnicas de aprendizagem supervisionada para antecipar ocorrências de sinistros viários visando aprimorar a gestão da segurança viária e na implementação de políticas públicas que subsidiem ações estratégicas de redução de sinistros. Nesse contexto, observa-se que os acidentes de trânsito são eventos influenciados por múltiplos fatores, como condições climáticas, infraestrutura viária, tipo do veículo dentre outros. Assim, na construção de um modelo preditivo eficiente, busca-se selecionar variáveis relevantes e algoritmos capazes de identificar padrões em dados históricos, de forma a melhorar a precisão das previsões e subsidiar ações estratégicas de segurança no trânsito.


**Palavras-Chave:** Previsão. Acidentes de Trânsito. Aprendizagem Supervisionada.

**Autores:** Gabriel Batista Pontes e Nercino José de Barros Neto

## Imports

In [173]:
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [174]:
import pickle

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

## Carregando o dataset

In [175]:
df = pd.read_csv('data/acidentes_pb.csv')

## Preparando os dados para o modelo

### Selecionando Xs

In [176]:
colunas_uteis = [
    'dia_semana',
    'br',
    'fase_dia',
    'condicao_metereologica',
    'classificacao_acidente',
    'pessoas',
    'veiculos',
]

df = df[colunas_uteis]

### Definindo o target(y)

In [177]:
df['com_vitima'] = df['classificacao_acidente'].apply(lambda x: False if x.lower() == 'sem vítimas' else True)
df.drop(columns=['classificacao_acidente'], inplace=True)

### Criando dummies

In [178]:
df.br = df.br.astype('category')
df = (
    df.pipe(pd.get_dummies, drop_first=True)
    .dropna()
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7800 entries, 0 to 7799
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype
---  ------                                   --------------  -----
 0   pessoas                                  7800 non-null   int64
 1   veiculos                                 7800 non-null   int64
 2   com_vitima                               7800 non-null   bool 
 3   dia_semana_quarta-feira                  7800 non-null   bool 
 4   dia_semana_quinta-feira                  7800 non-null   bool 
 5   dia_semana_segunda-feira                 7800 non-null   bool 
 6   dia_semana_sexta-feira                   7800 non-null   bool 
 7   dia_semana_sábado                        7800 non-null   bool 
 8   dia_semana_terça-feira                   7800 non-null   bool 
 9   br_101                                   7800 non-null   bool 
 10  br_104                                   7800 non-null   bool 
 11  br_1

### Tratando Xs numéricos

In [179]:
stats = {}

for col in ['pessoas', 'veiculos']:
    if col in df.columns and pd.api.types.is_integer_dtype(df[col]):
        min_val = df[col].min()
        max_val = df[col].max()
        stats[col] = [min_val, max_val]
        df[col] = (df[col] - min_val) / (max_val - min_val)

df.describe()

,pessoas,veiculos
count,7800.000000,7800.000000
mean,0.027640,0.037723
std,0.033839,0.042031
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.020408,0.047619
75%,0.040816,0.047619
max,1.000000,1.000000


## Criando o modelo

In [180]:
y = df['com_vitima']
X = df.drop(['com_vitima'], axis=1)

df['com_vitima'].value_counts()/len(df['com_vitima'])

com_vitima
True     0.818974
False    0.181026
Name: count, dtype: float64

### Definindo modelos para teste

In [181]:
models = {
    'Logit': LogisticRegression(),
    'NB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'Tree': DecisionTreeClassifier(),
    'Forest': RandomForestClassifier()
}

### Gerando os modelos

In [182]:
results = {}

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=30, scoring='accuracy')
    results[name] = {
        'model': model,
        'mean_accuracy': scores.mean(),
        'std_accuracy': scores.std()
    }

results

{'Logit': {'model': LogisticRegression(),
  'mean_accuracy': np.float64(0.8188461538461541),
  'std_accuracy': np.float64(0.001817621394712535)},
 'NB': {'model': GaussianNB(),
  'mean_accuracy': np.float64(0.22820512820512823),
  'std_accuracy': np.float64(0.019670193245139144)},
 'KNN': {'model': KNeighborsClassifier(),
  'mean_accuracy': np.float64(0.7946153846153848),
  'std_accuracy': np.float64(0.01824661217913558)},
 'Tree': {'model': DecisionTreeClassifier(),
  'mean_accuracy': np.float64(0.7765384615384614),
  'std_accuracy': np.float64(0.023716562676063608)},
 'Forest': {'model': RandomForestClassifier(),
  'mean_accuracy': np.float64(0.8011538461538463),
  'std_accuracy': np.float64(0.01757345811103427)}}

### Selecionando o melhor modelo

In [183]:
best_model_name = max(results, key=lambda name: results[name]['mean_accuracy'])
print(best_model_name)
best_model = results[best_model_name]['model']
best_model

Logit


LogisticRegression()

### Treinando o modelo

In [184]:
best_model.fit(X,y)

LogisticRegression()

### Salvando o modelo

In [185]:
r = {
    'metodo': best_model_name,
    'resultados': best_model,
    'escala': stats,
    'acuracia': best_model.score(X, y)
}
r

{'metodo': 'Logit',
 'resultados': LogisticRegression(),
 'escala': {'pessoas': [np.int64(1), np.int64(50)],
  'veiculos': [np.int64(1), np.int64(22)]},
 'acuracia': 0.8191025641025641}

In [186]:
filename = 'modelo_final.pkl'
pickle.dump(r, open(filename, "wb"))

## Pontos para revisão

- Separar o dataset em treino e teste
- Melhorar equilíbrio entre as classes no dataset
- Rever escolha de variáveis(Xs)
- Rever escolha de target(y)